In [ ]:
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, Robot
import cv2
import numpy as np
import time
from PID import PID

In [ ]:
camera_width = 320  # Camera resolution width
camera_height = 240  # Camera resolution height
forward_speed = 0.1  # Forward movement speed
turn_speed = 0.1  # Turning speed
turn_duration = 0.4  # Turn duration in seconds

steering_pid = PID(Kp=0.00001, Ki=0.0, Kd=0.00065, setpoint=camera_width / 2)  # PID for steering

# HSV thresholds for detection
lower_green = np.array([35, 60, 60])
upper_green = np.array([85, 255, 255])
lower_white = np.array([0, 0, 150])
upper_white = np.array([180, 60, 255])

camera = Camera.instance(width=camera_width, height=camera_height)  # Camera instance
robot = Robot()  # Robot instance

line_mask_widget = widgets.Image(format='jpeg', width=camera_width, height=camera_height)  # Display widget

# Robot control flags and buttons
robot_stop = True
stop_button = widgets.Button(description="STOP", button_style='danger', layout=widgets.Layout(width='100px', height='50px'))
start_button = widgets.Button(description="START", button_style='success', layout=widgets.Layout(width='100px', height='50px'))

def stop_robot(b):
    global robot_stop
    robot_stop = True
    robot.stop()  # Stop motors

def start_robot(b):
    global robot_stop
    robot_stop = False  # Allow motion

stop_button.on_click(stop_robot)
start_button.on_click(start_robot)

display(widgets.VBox([
    line_mask_widget,
    widgets.HBox([start_button, stop_button])
]))

state = "FOLLOW_LINE"  # Initial state
turn_start_time = None  # Timer for non-blocking turns
frame_counter = 0  # For visualization rate limiting

def follow_black_line(frame):
    global state, robot_stop, turn_start_time, frame_counter

    h, w, _ = frame.shape
    roi = frame[:, w//4: 3*w//4]  # Focus on middle vertical strip
    steering_pid.setpoint = roi.shape[1] / 2  # Center line target

    # ----------------- Preprocessing -----------------
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))  # Contrast enhancement
    gray = clahe.apply(gray)
    _, mask_black = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)  # Black line mask

    kernel = np.ones((3,3), np.uint8)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_CLOSE, kernel)  # Cleanup

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)  # Cleanup

    # ----------------- Turn detection -----------------
    turn_signal = None
    if np.any(mask_green):
        contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            x, y, gw, gh = cv2.boundingRect(cnt)
            white_below = False
            black_left = False
            black_right = False

            if y + gh > roi.shape[0] // 2:  # Only bottom half
                # White strip below green
                below_y1 = min(mask_white.shape[0]-1, y + gh)
                below_y2 = min(mask_white.shape[0], y + gh + 10)
                strip_below = mask_white[below_y1:below_y2, x:x+gw]
                white_below = np.mean(strip_below) > 128 if strip_below.size > 0 else False

                # Side black regions
                side_y1 = y
                side_y2 = y + gh
                left_x1 = max(0, x - gw)
                left_x2 = x
                right_x1 = x + gw
                right_x2 = min(mask_black.shape[1], x + 2 * gw)

                left_region = mask_black[side_y1:side_y2, left_x1:left_x2]
                right_region = mask_black[side_y1:side_y2, right_x1:right_x2]

                black_left = np.mean(left_region) > 128 if left_region.size > 0 else False
                black_right = np.mean(right_region) > 128 if right_region.size > 0 else False

                # Determine turn direction
                if white_below:
                    if black_right and not black_left:
                        turn_signal = "LEFT"
                    elif black_left and not black_right:
                        turn_signal = "RIGHT"

            # Draw green/red rectangle
            color = (0,255,0) if white_below else (0,0,255)
            cv2.rectangle(roi, (x, y), (x+gw, y+gh), color, 2)

    # ----------------- Line detection -----------------
    M = cv2.moments(mask_black)
    line_found = M["m00"] > 50  # Minimum area
    if line_found:
        cx_line = int(M["m10"] / M["m00"])
        cv2.circle(roi, (cx_line, h//2), 5, (0,0,255), -1)
        steering_pid.setpoint = w / 2  # Update PID setpoint

    # ----------------- Motor control -----------------
    left_motor = right_motor = 0

    if state == "FOLLOW_LINE":
        if line_found:
            steering = steering_pid.update(cx_line)
            left_motor = forward_speed - steering
            right_motor = forward_speed + steering
        if turn_signal == "LEFT":
            state = "TURN_LEFT"
            turn_start_time = time.time()
        elif turn_signal == "RIGHT":
            state = "TURN_RIGHT"
            turn_start_time = time.time()

    elif state.startswith("TURN_"):
        if turn_start_time is None:
            turn_start_time = time.time()
        elapsed = time.time() - turn_start_time
        if state == "TURN_LEFT":
            left_motor = -turn_speed
            right_motor = turn_speed
        else:  # TURN_RIGHT
            left_motor = turn_speed
            right_motor = -turn_speed
        if elapsed >= turn_duration:
            state = "FOLLOW_LINE"
            turn_start_time = None

    # Stop motors if required
    if robot_stop:
        left_motor = right_motor = 0

    robot.set_motors(left_motor, right_motor)

    # ----------------- Visualization -----------------
    frame_counter += 1
    if frame_counter % 3 == 0:
        debug_vis = cv2.addWeighted(roi, 0.8, cv2.cvtColor(mask_black, cv2.COLOR_GRAY2BGR), 0.3, 0)
        line_mask_widget.value = bgr8_to_jpeg(debug_vis)

# Attach callback
camera.observe(lambda change: follow_black_line(change['new']), names='value')

In [ ]:
robot.stop()
camera.stop()